In [1]:
import pandas as pd
import itertools
from datetime import date
from data_manager import DataManager
import analyze
from models import Player

dm = DataManager()

class Prop:
    def __init__(self, name, team, stat, threshold, odds, bet_type):
        self.name = name
        self.team = team
        self.stat = stat
        self.n = threshold
        self.odds = odds
        self.bet_type = bet_type
        self.probability = self.get_prop_probability()
        self.ev, self.house_prob = self.get_ev_and_implied_prob()
        self.print_out = f"""
            PLAYER: {self.name}
              STAT: {self.stat}
            THRESH: {self.n}
              ODDS: {self.odds}
              TYPE: {self.bet_type}
              PROB: {self.probability}
                EV: {self.ev}
        HOUSE_PROB: {self.house_prob}
            """
        print(self.print_out)
        self.entry = {
            "PLAYER": self.name,
              "TEAM": self.team,
              "STAT": self.stat,
            "THRESH": self.n,
              "ODDS": self.odds,
              "TYPE": self.bet_type,
              "PROB": self.probability,
                "EV": self.ev,
        "HOUSE_PROB": self.house_prob
        }
    
    def get_prop_probability(self, last_n_games=25):
        player_id = dm.get_player_id(self.name)
        data = dm.get_and_save_player_data(player_id, self.name).sort_values(by='date', ascending=False).head(last_n_games).copy()
        # print(data.head())
        if self.bet_type == "over":
            return analyze.estimate_probability_poisson_over(data, self.stat, self.n)
        elif self.bet_type == "under":
            return analyze.estimate_probability_poisson_under(data, self.stat, self.n)
        else:
            raise ValueError("Invalid bet type. Use 'over' or 'under'.")
        
    def get_ev_and_implied_prob(self):
        odds = self.american_to_decimal(self.odds)
        house_probability = analyze.estimate_implied_probability(odds)
        ev = analyze.calculate_ev(self.probability, odds, 5)
        return ev, house_probability
    
    @staticmethod
    def american_to_decimal(american_odds):
        """Convert American odds to decimal odds."""
        if american_odds > 0:
            return 1 + (american_odds / 100)
        else:
            return 1 + (100 / abs(american_odds))
    

def extract_raw_data(file_path): # .csv
    # gets input from A1
    # Sample input text (use the content of your file here)
    raw_input = pd.read_csv(file_path)
    list_of_raw_input = list(raw_input.iloc[:, 0])
    return list_of_raw_input


def load_available_props():
    
    raw_input = extract_raw_data("prop_lines/prop_lines.csv")
    print("Received raw input.")
    stat_names = {
             'PointsSGP': "points",
            'AssistsSGP': "assists",
        'Threes MadeSGP': "fg3m",
           'ReboundsSGP': "rebounds",
   'Field Goals MadeSGP': "fgm",
             'StealsSGP': "steals",
             'BlocksSGP': "blocks",
        }
    #debug stat_name_inputs = extract_raw_data("prop_lines/player_prop_categories.csv")
    players = dm.query_players()
    player_names = [player.name for player in players]
    teams = dm.query_teams()
    team_names = [team.nickname for team in teams]
    row_of_interest = 0
    current_category = None
    current_player = None
    current_team = None
    records = []
    for _, item in enumerate(raw_input):
        if item in stat_names:
            current_category = stat_names[item]
            print(f"Loading {current_category} props.")
        if item != current_team:
            if item in team_names:
                current_team = item
        if item in player_names:
            current_player = item 
            assert current_team
            record = [current_player, current_team, current_category]
            row_of_interest = 6


        if row_of_interest:
            row_of_interest -= 1
            if row_of_interest < 5:
                record.append(item)
                if row_of_interest == 1:
                    records.append(record)
                    record = []
    df = pd.DataFrame.from_records(records, columns=["player_name", "team", "stat", "over_threshold", "over_odds", "under_threshold", "under_odds"])
    
    df['player_name'] = df['player_name'].astype(str)
    df['team'] = df['team'].astype(str)
    df['stat'] = df['stat'].astype(str)
    df['over_threshold'] = df['over_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['under_threshold'] = df['under_threshold'].str.extract(r'(\d+\.\d+)').astype(float)
    df['over_odds'] = df['over_odds'].astype(int)
    df['under_odds'] = df['under_odds'].astype(int)

    return df


def get_analyzed_props(available_props):
    props = []
    for _, row in available_props.iterrows():
        for bet_type in ["over", "under"]:
            prop = Prop(
                    name=row["player_name"], 
                    team=row["team"],
                    stat=row["stat"], 
                threshold=row[f"{bet_type}_threshold"], 
                    odds=row[f"{bet_type}_odds"], 
                bet_type=bet_type
                )
            props.append(prop)    
            print("Prop object created.")

    return props


def generate_heterogenous_combinations(df, n):

    # Generate all combinations of n rows
    combinations = list(itertools.combinations(df.index, n))

    # Function to evaluate heterogeneity of a combination
    def evaluate_heterogeneity(comb, df):
        comb_list = list(comb)
        players = df.loc[comb_list, 'PLAYER']
        stats = df.loc[comb_list, 'STAT']
        teams = df.loc[comb_list, 'TEAM']
        # Calculate a simple heterogeneity score (you can define your own logic)
        player_score = len(set(players))
        stat_score = len(set(stats))
        team_score = len(set(teams))
        return player_score + stat_score + team_score

    # Evaluate all combinations and sort them by heterogeneity score
    comb_scores = [(comb, evaluate_heterogeneity(comb, df)) for comb in combinations]
    comb_scores_sorted = sorted(comb_scores, key=lambda x: x[1], reverse=True)

    # Select the most heterogeneous combinations (you can define how many you want)
    top_combinations = comb_scores_sorted # Top 5 combinations for example

    # Display the most heterogeneous combinations
    for comb, score in top_combinations:
        print(f"Combination: {comb}, Score: {score}")
        print(df.loc[list(comb)])
        print()

    # Optional: Convert combinations to DataFrame
    top_comb_dfs = [(df.loc[list(comb)], score) for comb, score in top_combinations]

    return top_comb_dfs


In [2]:
available_props = load_available_props()
analyzed_props = get_analyzed_props(available_props)
print(f"num of props: {len(analyzed_props)}")
profitable_props = [prop for prop in analyzed_props if prop.ev > 0]
print(f"num of profitable props: {len(profitable_props)}")
profitable_props_df = pd.DataFrame.from_dict([prop.entry for prop in profitable_props])


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.

            PLAYER: Jalen Brunson
              STAT: points
            THRESH: 34.5
              ODDS: -115
              TYPE: over
              PROB: 0.44909906446823733
                EV: -0.8019000495360427
        HOUSE_PROB: 0.5348837209302326
            
Prop object created.

            PLAYER: Jalen Brunson
              STAT: points
            THRESH: 34.5
              ODDS: -115
              TYPE: under
              PROB: 0.5509009355317627
                EV: 0.14972613649256417
        HOUSE_PROB: 0.5348837209302326
            
Prop object created.

            PLAYER: Josh Hart
              STAT: points
            THRESH: 14.5
              ODDS: -125
              TYPE: over
              PROB: 0.32895814554194647
                EV: -2.0393766901224817
        HOUSE_PROB: 0.5555555555555

In [3]:
top_ev = profitable_props_df.sort_values(by="PROB", ascending=False).head(24)
parlays = generate_heterogenous_combinations(profitable_props_df, 6)

In [ ]:
candidates = []

for i, parlay in enumerate(parlays):
    df, score = parlay
    today = date.today()
    
    probs = list(df['PROB'])
    odds = [odds for odds in list(df['ODDS'])]
    combined_ev = analyze.analyze_parlay(probs, odds, 5)
    combined_prob = analyze.calculate_combined_probability(probs)
    df['COMBINED EV'] = combined_ev
    df['COMBINED PROB'] = combined_prob

    parlay_tag = f"{today}_{i}_"
    candidate = [parlay_tag]
    props = []
    for _, row in df.iterrows():
        prop_info = f"{row['PLAYER']}_{row['STAT']}_{row['THRESH']}_{row['ODDS']}_{row['TYPE']}_P{row['PROB']:.3f}_EV{row['EV']:.3f}_HOUSE_PROV{row['HOUSE_PROB']:.3f}"
        props.append(prop_info)
    candidate.append(" + ".join(props))
    candidate += [combined_prob, combined_ev, score]
    candidates.append(candidate)


In [ ]:
candidates_df = pd.DataFrame.from_records(candidates, columns = ["parlay_tag", "props", "COMBINED_PROB", "COMBINED_EV", "HETERO"])
candidates_df.to_csv(f"prop_lines/{date.today()}_candidates_7props.csv")